In [1]:
import json
import gzip
import re
from collections import Counter
from datetime import datetime, timedelta
import pytz

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
example_pid = '24450f28-a039-57d8-95c9-d7ba5508ecd4' # this patient has been admitted 3 times
# check the Query Encounter block, at the bottom, you can see patient_id + admission times

MimicPatient_path = '/content/drive/MyDrive/MIMIC4_FHIR/MimicPatient.ndjson.gz' # patient
MimicEncounter_path = '/content/drive/MyDrive/MIMIC4_FHIR/MimicEncounter.ndjson.gz' # admission
MimicObservationLabevents_path = '/content/drive/MyDrive/MIMIC4_FHIR/MimicObservationLabevents.ndjson.gz' # lab event
MimicCondition_path = '/content/drive/MyDrive/MIMIC4_FHIR/MimicCondition.ndjson.gz' # diagnosis
MimicProcedure_path = '/content/drive/MyDrive/MIMIC4_FHIR/MimicProcedure.ndjson.gz' # procedure
MimicMedicationAdministration_path = '/content/drive/MyDrive/MIMIC4_FHIR/MimicMedicationAdministration.ndjson.gz' # medication

## Query Patient Information (example code from PhysioNet)

In [4]:
def extract_demographics(patient): # Example code from physioNet
    demographics = {
        'id': patient['id'],
        'gender': patient.get('gender'),
        'birth_date': patient.get('birthDate'),
        'marital_status': None,
        'race': None,
        'ethnicity': None
    }
    # Extract marital status
    marital = patient.get('maritalStatus', {}).get('coding', [])
    if marital:
        demographics['marital_status'] = marital[0].get('display') or marital[0].get('code')

    # Extract race and ethnicity from extensions
    for ext in patient.get('extension', []):
        url = ext.get('url', '')
        if 'race' in url:
            for race_ext in ext.get('extension', []):
                if race_ext.get('url') == 'text':
                    demographics['race'] = race_ext.get('valueString')
        elif 'ethnicity' in url:
            for eth_ext in ext.get('extension', []):
                if eth_ext.get('url') == 'text':
                    demographics['ethnicity'] = eth_ext.get('valueString')
    return demographics

def load_mimic_patient(patient_file):
    """Extract demographics from all patients"""
    data = []

    with gzip.open(patient_file, 'rt') as f:
        for line in f:
            patient = json.loads(line.strip())
            if patient:
                demographics = extract_demographics(patient)
                data.append(demographics)

    return data

patient_file = MimicPatient_path
patients = load_mimic_patient(patient_file)
print(f"Loaded {len(patients)} patients")

Loaded 100 patients


In [5]:
next((p for p in patients if p["id"] == example_pid), None)

{'id': '24450f28-a039-57d8-95c9-d7ba5508ecd4',
 'gender': 'female',
 'birth_date': '2119-12-30',
 'marital_status': 'D',
 'race': 'White',
 'ethnicity': 'Not Hispanic or Latino'}

## Query Encounter
* FHIR table: MimicEncounter.ndjson.gz
* MIMIC4 Demo table: admissions

#### Columns:
1. subject_id: Encounter.subject
2. hadm_id: Encounter.id + Encounter.identifier.value
3. admittime: Encounter.period.start
4. dischtime: Encounter.period.end
5. admission_type: Encounter.priority + Encounter.class
    * explanation: [class](https://terminology.hl7.org/6.5.0/ValueSet-encounter-class.html) - classification of patient encounter context
      * IMP: inpatient encounter
      * AMB: ambulatory
      * OBSENC: observation encounter
      * EMER: emergency
      * VR: virtual
      * HH: home health
    * priority: A code or set of codes (e.g., for routine, emergency,) specifying the urgency under which the Act happened, can happen, is happening, is intended to happen, or is requested/demanded to happen.
      * I find it hard to understand how emergent they are based on their name, feel free to check the webpage linked [here](https://terminology.hl7.org/6.5.0/ValueSet-v3-ActPriority.html).

In [6]:
def extract_encounter(encounter):
    encounter_info = {
        'hadm_id': encounter.get('id'),
        'hadm_identifier': None,
        'subject_id': encounter.get('subject', {}).get('reference', '').replace('Patient/', ''),
        'admittime': None,
        'dischtime': None,
        #'admittime': encounter.get('period', {}).get('start'),
        # 'dischtime': encounter.get('period', {}).get('end'),

        'class': encounter.get('class', {}).get('code'),
        'priority': encounter.get('priority', {}).get('coding', [{}])[0].get('code')
    }

    # extract identifier value (alternative hadm_id)
    identifiers = encounter.get('identifier', [])
    if identifiers:
        encounter_info['hadm_identifier'] = identifiers[0].get('value')

    eastern = pytz.timezone('US/Eastern')

    admittime_str = encounter.get('period', {}).get('start')
    if admittime_str:
        dt = datetime.fromisoformat(admittime_str)
        encounter_info['admittime'] = dt.astimezone(eastern).strftime('%Y-%m-%d %H:%M:%S') # this time change from UTC is not necessary
    # because MIMIC4 still perserves the original order of data, I converted them back to EST for easier readability

    dischtime_str = encounter.get('period', {}).get('end')
    if dischtime_str:
        dt = datetime.fromisoformat(dischtime_str)
        encounter_info['dischtime'] = dt.astimezone(eastern).strftime('%Y-%m-%d %H:%M:%S')

    return encounter_info

def load_mimic_encounters(encounter_file):
    """Extract encounter info from all records"""
    data = []
    with gzip.open(encounter_file, 'rt') as f:
        for line in f:
            if line.strip():
                encounter = json.loads(line.strip())
                if encounter:
                    encounter_info = extract_encounter(encounter)
                    data.append(encounter_info)
    return data

# Usage
encounter_file = MimicEncounter_path
encounters = load_mimic_encounters(encounter_file)
print(f"Loaded {len(encounters)} encounter records")

Loaded 275 encounter records


In [7]:
# example: Get all encounters for a patient (example_pid)
patient_encounters = [e for e in encounters if e["subject_id"] == example_pid]
print(f"\nFound {len(patient_encounters)} encounters for patient {example_pid}")

for enc in patient_encounters:
    print(enc)


Found 3 encounters for patient 24450f28-a039-57d8-95c9-d7ba5508ecd4
{'hadm_id': '0a83714b-0c0e-5761-b9ea-3ff6d5fd5c8b', 'hadm_identifier': '22196214', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'admittime': '2148-03-07 23:30:00', 'dischtime': '2148-03-13 14:50:00', 'class': 'OBSENC', 'priority': 'R'}
{'hadm_id': '0071a339-74cd-596a-9083-771d41d6d118', 'hadm_identifier': '22429197', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'admittime': '2147-12-30 08:40:00', 'dischtime': '2148-01-11 17:55:00', 'class': 'EMER', 'priority': 'EM'}
{'hadm_id': '7e8c1a0f-0a7b-5b91-a351-0a10939deace', 'hadm_identifier': '22950920', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'admittime': '2148-01-25 22:58:00', 'dischtime': '2148-01-30 11:23:00', 'class': 'OBSENC', 'priority': 'R'}


In [8]:
# getting a list of patient_id with their admission times
admission_counts = Counter(e['subject_id'] for e in encounters)
patient_admissions = admission_counts.most_common()

print(f"Total unique patients: {len(patient_admissions)}")
print(f"\nTop 20 patients by admission count:")
for patient_id, count in patient_admissions[:20]:
    print(f"Patient {patient_id}: {count} admissions")

patient_admission_list = [
    {'subject_id': patient_id, 'admission_count': count}
    for patient_id, count in patient_admissions
]

Total unique patients: 100

Top 20 patients by admission count:
Patient cb70e6ae-90b1-562b-8ab0-467c65d18d5e: 20 admissions
Patient 1cf9e585-806c-513b-80af-4ca565a28231: 13 admissions
Patient a5d4cb17-db8d-574b-bd88-71473088fd9a: 12 admissions
Patient 8e77dd0b-932d-5790-9ba6-5c6df8434457: 10 admissions
Patient adde1635-3110-5e92-b9f0-7a6d845a1784: 10 admissions
Patient fa5fbf9c-23e3-5ef3-9cfb-24d20a950314: 10 admissions
Patient 4f773083-7f4d-5378-b839-c24ca1e15434: 9 admissions
Patient 91f25704-6153-5259-bdd7-2ca6478de14a: 8 admissions
Patient 0c2243d2-987b-5cbd-8eb1-170a80647693: 7 admissions
Patient 77e10fd0-6a1c-5547-a130-fae1341acf36: 7 admissions
Patient 9c3ebb7e-d087-519e-bea4-31c3d4aac7ff: 7 admissions
Patient e1de99bc-3bc5-565e-9ee6-69675b9cc267: 7 admissions
Patient 4365e125-c049-525a-9459-16d5e6947ad2: 6 admissions
Patient b410dd44-7d65-56f9-974f-2751e8aa80e2: 6 admissions
Patient f77a5b72-65fd-5b20-8cef-6b6be4791265: 6 admissions
Patient 28dcf33b-0c52-587f-83ad-2a3270976719:

## Query ObservationLabevents   
* FHIR table: MimicObservationLabevents      
* MIMIC4 demo table: labevents + d_labitems

#### Columns
1. labevent_id:
2. subject_id: Observation.subject
3. hadm_id: Observation.encounter
4. value: Observation.valueString (Only used if MIMIC-IV valuenum is NULL)
5. valuenum: Observation.valueQuantity.value
6. valueuom: Observation.valueQuantity.unit
7. ref_range_lower: Observation.referenceRange.low.value
8. ref_range_upper: Observation.referenceRange.high.value
9. flag: Observation.interpretation
10. itemid: Observation.code.coding.code
11. label: Observation.code.coding.display

* Claude gave me more columns than I needed so I kept them anyway :)

In [9]:
def extract_labevent(observation):
    labevent_info = {
        'labevent_id': observation.get('id'),
        'subject_id': observation.get('subject', {}).get('reference', '').replace('Patient/', ''),
        'hadm_id': observation.get('encounter', {}).get('reference', '').replace('Encounter/', ''),
        'specimen_id': observation.get('specimen', {}).get('reference', '').replace('Specimen/', ''),
        'itemid': None,
        'label': None,
        # 'charttime': observation.get('effectiveDateTime'),
        # 'storetime': observation.get('issued'),
        'charttime': None,
        'storetime': None,

        'valuenum': None,
        'valueuom': None,
        'ref_range_lower': None,
        'ref_range_upper': None,
        'flag': None,
        'comments': None
    }

    coding = observation.get('code', {}).get('coding', []) # lab test code and label
    if coding:
        labevent_info['itemid'] = coding[0].get('code')
        labevent_info['label'] = coding[0].get('display')

    value_quantity = observation.get('valueQuantity', {}) # value, (prioritize valueQuantity, fallback to valueString)
    if value_quantity.get('value') is not None:
        labevent_info['valuenum'] = value_quantity.get('value')
        labevent_info['valueuom'] = value_quantity.get('unit')
    else:

        labevent_info['valuenum'] = observation.get('valueString') # only use valueString if valueQuantity is NULL by doc

    ref_range = observation.get('referenceRange', [])
    if ref_range:
        labevent_info['ref_range_lower'] = ref_range[0].get('low', {}).get('value')
        labevent_info['ref_range_upper'] = ref_range[0].get('high', {}).get('value')

    interpretation = observation.get('interpretation', [])
    if interpretation:
        interp_coding = interpretation[0].get('coding', [])
        if interp_coding:
            labevent_info['flag'] = interp_coding[0].get('code')

    notes = observation.get('note', [])
    if notes:
        labevent_info['comments'] = notes[0].get('text')

    eastern = pytz.timezone('US/Eastern')

    charttime_str = observation.get('effectiveDateTime')
    if charttime_str:
        dt = datetime.fromisoformat(charttime_str)
        labevent_info['charttime'] = dt.astimezone(eastern).strftime('%Y-%m-%d %H:%M:%S')

    storetime_str = observation.get('issued')
    if storetime_str:
        dt = datetime.fromisoformat(storetime_str)
        labevent_info['storetime'] = dt.astimezone(eastern).strftime('%Y-%m-%d %H:%M:%S')

    return labevent_info

def load_mimic_labevents(labevent_file):
    """Extract lab event info from all records"""
    data = []
    with gzip.open(labevent_file, 'rt') as f:
        for line in f:
            if line.strip():
                observation = json.loads(line.strip())
                if observation:
                    labevent_info = extract_labevent(observation)
                    data.append(labevent_info)
    return data

# Usage
labevent_file = "/content/drive/MyDrive/MIMIC4_FHIR/MimicObservationLabevents.ndjson.gz"
labevents = load_mimic_labevents(labevent_file)
print(f"Loaded {len(labevents)} lab event records")

Loaded 107727 lab event records


In [10]:
patient_labs = [l for l in labevents if l["subject_id"] == example_pid]
print(f"\nFound {len(patient_labs)} lab events for patient {example_pid}")
print("\nFirst 5 lab events:")
for lab in patient_labs[:5]:
    print(lab)


Found 693 lab events for patient 24450f28-a039-57d8-95c9-d7ba5508ecd4

First 5 lab events:
{'labevent_id': 'd92402f8-b53c-51c6-8944-7a6fb42718e3', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hadm_id': '0071a339-74cd-596a-9083-771d41d6d118', 'specimen_id': '62bb5547-c22f-52b3-9433-7ffc4bfc2b7a', 'itemid': '51466', 'label': 'Blood', 'charttime': '2148-01-10 03:00:00', 'storetime': '2148-01-10 04:36:00', 'valuenum': 'SM .', 'valueuom': None, 'ref_range_lower': None, 'ref_range_upper': None, 'flag': None, 'comments': 'SM .'}
{'labevent_id': '77662479-2c29-5b48-8161-3f7fcbcb091a', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hadm_id': '0071a339-74cd-596a-9083-771d41d6d118', 'specimen_id': '62bb5547-c22f-52b3-9433-7ffc4bfc2b7a', 'itemid': '51478', 'label': 'Glucose', 'charttime': '2148-01-10 03:00:00', 'storetime': '2148-01-10 04:36:00', 'valuenum': 'NEG.', 'valueuom': None, 'ref_range_lower': None, 'ref_range_upper': None, 'flag': None, 'comments': 'NEG.'}
{'labevent_i

## Query Procedure
* FHIR table: MimicProcedure.ndjson.gz
* MIMIC4 Demo table: procedures_icd + d_icd_procedures

#### Columns
1. subject_id: Procedure.subject
2. hadm_id: Procedure.encounter
3. chartdate: Procedure.performedDateTime
4. icd_code: Procedure.code.coding.code
5. icd_version: Procedure.code.coding.system (may not include this)
6. long_title: Procedure.code.coding.display

In [11]:
def extract_procedure(procedure):
    procedure_info = {
        'procedure_id': procedure.get('id'),
        'subject_id': procedure.get('subject', {}).get('reference', '').replace('Patient/', ''),
        'hadm_id': procedure.get('encounter', {}).get('reference', '').replace('Encounter/', ''),
        # 'chartdate': procedure.get('performedDateTime'),
        'chartdate': None,
        'icd_code': None,
        'long_title': None
    }

    # Extract ICD code and description
    coding = procedure.get('code', {}).get('coding', [])
    if coding:
        procedure_info['icd_code'] = coding[0].get('code')
        procedure_info['long_title'] = coding[0].get('display')

    eastern = pytz.timezone('US/Eastern')

    chartdate_str = procedure.get('performedDateTime')
    if chartdate_str:
        dt = datetime.fromisoformat(chartdate_str)
        procedure_info['chartdate'] = dt.astimezone(eastern).strftime('%Y-%m-%d %H:%M:%S')

    return procedure_info

def load_mimic_procedures(procedure_file):
    """Extract procedure info from all records"""
    data = []
    with gzip.open(procedure_file, 'rt') as f:
        for line in f:
            if line.strip():
                procedure = json.loads(line.strip())
                if procedure:
                    procedure_info = extract_procedure(procedure)
                    data.append(procedure_info)
    return data

# Usage
procedure_file = MimicProcedure_path
procedures = load_mimic_procedures(procedure_file)
print(f"Loaded {len(procedures)} procedure records")

Loaded 722 procedure records


In [12]:
# get all procedures for a patient
patient_id = "a87c1099-8c87-5548-8fc0-56972a82cebf"
patient_procedures = [p for p in procedures if p["subject_id"] == patient_id]
print(f"\nFound {len(patient_procedures)} procedures for patient {patient_id}")
print("\nFirst 5 procedure records:")
for proc in patient_procedures:
    print(proc)


Found 5 procedures for patient a87c1099-8c87-5548-8fc0-56972a82cebf

First 5 procedure records:
{'procedure_id': '65969217-dd0b-55de-b586-991aa6205642', 'subject_id': 'a87c1099-8c87-5548-8fc0-56972a82cebf', 'hadm_id': 'ac3f7b4a-3199-5277-907e-d2b1b411a818', 'chartdate': '2131-02-27 00:00:00', 'icd_code': '9604', 'long_title': 'Insertion of endotracheal tube'}
{'procedure_id': '112e076a-6c25-54c4-abe3-77240d566940', 'subject_id': 'a87c1099-8c87-5548-8fc0-56972a82cebf', 'hadm_id': 'ac3f7b4a-3199-5277-907e-d2b1b411a818', 'chartdate': '2131-02-27 00:00:00', 'icd_code': '3323', 'long_title': 'Other bronchoscopy'}
{'procedure_id': 'f7cb1850-033b-51fc-878c-f325dcc1217e', 'subject_id': 'a87c1099-8c87-5548-8fc0-56972a82cebf', 'hadm_id': 'ac3f7b4a-3199-5277-907e-d2b1b411a818', 'chartdate': '2131-02-27 00:00:00', 'icd_code': '3893', 'long_title': 'Venous catheterization, not elsewhere classified'}
{'procedure_id': 'f1a77feb-5cfb-524c-9138-9bb5a37d1f24', 'subject_id': 'a87c1099-8c87-5548-8fc0-569

## Query Condition

* FHIR table: MimicCondition.ndjson.gz
* MIMIC4 Demo table: diagnoses_icd + d_icd_diagnoses

#### Columns
1. subject_id: Condition.subject
2. hadm_id: Condition.encounter
3. icd_code: Condition.code.coding.code
4. long_title: Condition.code.coding.display

In [13]:
def extract_condition(condition):
    condition_info = {
        'condition_id': condition.get('id'),
        'subject_id': condition.get('subject', {}).get('reference', '').replace('Patient/', ''),
        'hadm_id': condition.get('encounter', {}).get('reference', '').replace('Encounter/', ''),
        'icd_code': None,
        'long_title': None
    }

    # extract ICD code and description
    coding = condition.get('code', {}).get('coding', [])
    if coding:
        condition_info['icd_code'] = coding[0].get('code')
        condition_info['long_title'] = coding[0].get('display')

    return condition_info

def load_mimic_conditions(condition_file):
    """Extract condition info from all records"""
    data = []
    with gzip.open(condition_file, 'rt') as f:
        for line in f:
            if line.strip():
                condition = json.loads(line.strip())
                if condition:
                    condition_info = extract_condition(condition)
                    data.append(condition_info)
    return data

# Usage
condition_file = MimicCondition_path
conditions = load_mimic_conditions(condition_file)
print(f"Loaded {len(conditions)} condition records")

Loaded 4506 condition records


In [14]:
patient_conditions = [c for c in conditions if c["subject_id"] == example_pid]
print(f"\nFound {len(patient_conditions)} diagnoses for patient {example_pid}")

for cond in patient_conditions[:10]:
    print(cond)


Found 49 diagnoses for patient 24450f28-a039-57d8-95c9-d7ba5508ecd4
{'condition_id': '352f13d2-d630-5509-808b-1d78b57edafb', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hadm_id': '0a83714b-0c0e-5761-b9ea-3ff6d5fd5c8b', 'icd_code': 'I272', 'long_title': 'Other secondary pulmonary hypertension'}
{'condition_id': '3a2556f5-584e-55ba-8f1f-ce8096d2194d', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hadm_id': '0a83714b-0c0e-5761-b9ea-3ff6d5fd5c8b', 'icd_code': 'F1210', 'long_title': 'Cannabis abuse, uncomplicated'}
{'condition_id': '52399795-15ef-57a5-875c-49a791524933', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hadm_id': '0071a339-74cd-596a-9083-771d41d6d118', 'icd_code': 'J9600', 'long_title': 'Acute respiratory failure, unspecified whether with hypoxia or hypercapnia'}
{'condition_id': 'c2cb284b-3a9e-5a3b-a9d4-559e7c8dc59b', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hadm_id': '0071a339-74cd-596a-9083-771d41d6d118', 'icd_code': 'S240XXA', 'lo

## Query MedicationAdministration
* FHIR table: MimicMedicationAdministration.ndjson.gz
* MIMIC4 Demo table: emar, emar_details
* [Link](https://mimic.mit.edu/fhir/mimic-fhir-mappings.html#medicationadministration) to mapping directory

#### Columns
1. subject_id: MedicationAdministration.subject
2. hadm_id: MedicationAdministration.context
3. emar_id: MedicationAdministration.id
4. medication: MedicationAdministration.medicationCodeableConcept
5. event_txt: MedicationAdministration.route, this one is about the administration. Most frequently event_txt is ‘Administered’, but other possible values are ‘Applied’, ‘Confirmed’, ‘Delayed’, ‘Not Given’, and so on. (from emar_detail)
6. route: MedicationAdministration.route (from emar_detail)



In [15]:
def extract_medication(medication):
    medication_info = {
        'emar_id': medication.get('id'),
        'subject_id': medication.get('subject', {}).get('reference', '').replace('Patient/', ''),
        'hospitalization_id': medication.get('context', {}).get('reference', '').replace('Encounter/', ''),
        'medication': None,
        'status': medication.get('status'),
        # 'charttime': medication.get('effectiveDateTime')
        'charttime': None
    }

    eastern = pytz.timezone('US/Eastern')

    charttime_str = medication.get('effectiveDateTime')
    if charttime_str:
        dt = datetime.fromisoformat(charttime_str)
        medication_info['charttime'] = dt.astimezone(eastern).strftime('%Y-%m-%d %H:%M:%S')

    # extract medication code from medicationCodeableConcept
    med_codeable = medication.get('medicationCodeableConcept', {}).get('coding', [])
    if med_codeable:
        medication_info['medication'] = med_codeable[0].get('code')

    return medication_info

def load_mimic_medications(medication_file):
    """Extract medication info from all records"""
    data = []
    with gzip.open(medication_file, 'rt') as f:
        for line in f:
            if line.strip():
                medication = json.loads(line.strip())
                if medication:
                    med_info = extract_medication(medication)
                    data.append(med_info)
    return data

# Usage
medication_file = MimicMedicationAdministration_path
medications = load_mimic_medications(medication_file)
print(f"Loaded {len(medications)} medication records")

Loaded 36131 medication records


In [16]:
# Retrive medication record based on a patient_id
patient_meds = [m for m in medications if m["subject_id"] == example_pid]


print(f"Found {len(patient_meds)} medication administrations for patient {example_pid}")

for med in patient_meds[:10]:
    print(med)

Found 888 medication administrations for patient 24450f28-a039-57d8-95c9-d7ba5508ecd4
{'emar_id': 'abd7d371-9ce9-529a-861f-fb233f8be6b0', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hospitalization_id': '0a83714b-0c0e-5761-b9ea-3ff6d5fd5c8b', 'medication': 'NACLFLUSH', 'status': 'completed', 'charttime': '2148-03-12 12:23:00'}
{'emar_id': 'd290a850-63f2-5ff7-bd8f-78f5e7a0e85d', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hospitalization_id': '0071a339-74cd-596a-9083-771d41d6d118', 'medication': 'FURO20I', 'status': 'completed', 'charttime': '2148-01-06 18:27:00'}
{'emar_id': 'dd06b24a-2098-5d25-a608-bf220eaa71ed', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hospitalization_id': '0071a339-74cd-596a-9083-771d41d6d118', 'medication': 'ERY25010NS', 'status': 'completed', 'charttime': '2148-01-05 06:28:00'}
{'emar_id': '2bf052ea-cb34-525c-840d-b193b69444eb', 'subject_id': '24450f28-a039-57d8-95c9-d7ba5508ecd4', 'hospitalization_id': '7e8c1a0f-0a7b-5b91-a351-

In [17]:
def get_top_medications(patient_meds, years=None):
    """Get frequency table of medications for a patient"""
    filtered_meds = patient_meds

    if years:
        # find the most recent charttime because we have de-identified time
        valid_times = [m['charttime'] for m in patient_meds if m['charttime']]
        if valid_times:
            max_time = max(datetime.strptime(t, '%Y-%m-%d %H:%M:%S') for t in valid_times)
            cutoff_date = max_time - timedelta(days=years*365)

            filtered_meds = [
                m for m in patient_meds
                if m['charttime'] and datetime.strptime(m['charttime'], '%Y-%m-%d %H:%M:%S') >= cutoff_date
            ]

    med_counts = Counter(m['medication'] for m in filtered_meds if m['medication'])
    return med_counts.most_common(5)


print("\n--- Top 5 Medications (All Time) ---")
top_meds = get_top_medications(patient_meds)
for med, count in top_meds:
    print(f"{med}: {count} administrations")

print("\n--- Top 5 Medications (Past 2 Years from Most Recent) ---")
top_meds_2y = get_top_medications(patient_meds, years=2)
for med, count in top_meds_2y:
    print(f"{med}: {count} administrations")


--- Top 5 Medications (All Time) ---
OXYCO5L: 62 administrations
HYDR1SYR: 58 administrations
NACLFLUSH: 48 administrations
HYDR2: 43 administrations
HEPA5I: 42 administrations

--- Top 5 Medications (Past 2 Years from Most Recent) ---
OXYCO5L: 62 administrations
HYDR1SYR: 58 administrations
NACLFLUSH: 48 administrations
HYDR2: 43 administrations
HEPA5I: 42 administrations


## Get one patient information
I hope this would be sufficient to be prompted to LLM for analyzing the patient history, at least enough for the design review purpose.

I'll definitely look into more about what physicians actually want and what they look at in EHR in real life.

1. patient_info [patients]
2. admission [patient_encounters]
3. lab results [labevents]
4. procedures [procedures]
5. diagnosis [conditions]
6. medication [patient_meds]

In [18]:
example_pid = '24450f28-a039-57d8-95c9-d7ba5508ecd4'

print("=" * 100)
print(f"PATIENT SUMMARY REPORT")
print(f"Patient ID: {example_pid}")
print("=" * 80)

print("\n1. PATIENT DEMOGRAPHICS")
print("-" * 80)
patient_info = next((p for p in patients if p["id"] == example_pid), None)
if patient_info:
    for key, value in patient_info.items():
        print(f"{key}: {value}")
else:
    print("Patient not found")

print("\n2. ADMISSIONS")
print("-" * 80)
patient_encounters = [e for e in encounters if e["subject_id"] == example_pid]
print(f"Total admissions: {len(patient_encounters)}\n")
for i, enc in enumerate(patient_encounters, 1):
    print(f"Admission {i}:")
    print(f"  hadm_id: {enc['hadm_id']}")
    print(f"  admittime: {enc['admittime']}")
    print(f"  dischtime: {enc['dischtime']}")
    print(f"  class: {enc['class']}, priority: {enc['priority']}")

print("\n3. LAB RESULTS (Past 2 Years - Top 10)")
print("-" * 80)
patient_labs = [l for l in labevents if l["subject_id"] == example_pid]
print(f"Total lab events: {len(patient_labs)}")

valid_labs = [l for l in patient_labs if l['charttime']]
if valid_labs:
    max_time = max(datetime.strptime(l['charttime'], '%Y-%m-%d %H:%M:%S') for l in valid_labs)
    cd = max_time - timedelta(days=2 * 365)
    recent_labs = [l for l in valid_labs if datetime.strptime(l['charttime'], '%Y-%m-%d %H:%M:%S') >= cd]

    lab_counts = Counter(l['label'] for l in recent_labs if l['label'])
    total_recent = len(recent_labs)

    print(f"Lab events in past 2 years: {total_recent}\n")
    print("Top 10 most frequent labs (with latest result) in past 2 years:")
    for i, (lab_name, count) in enumerate(lab_counts.most_common(10), 1):
        pct = (count / total_recent * 100) if total_recent > 0 else 0
        lab_results = [l for l in recent_labs if l['label'] == lab_name]
        latest = max(lab_results, key=lambda x: x['charttime'])
        unit = latest['valueuom'] if latest['valueuom'] else ''
        value_str = f"{latest['valuenum']} {unit}".strip()
        print(f"{i}. {lab_name}: {count} times ({pct:.1f}%) - Latest: {value_str} [{latest['charttime']}]")

    print("\nMost recent 10 lab results:")
    sorted_labs = sorted(recent_labs, key=lambda x: x['charttime'], reverse=True)
    for i, lab in enumerate(sorted_labs[:10], 1):
        unit = lab['valueuom'] if lab['valueuom'] else ''
        value_str = f"{lab['valuenum']} {unit}".strip()
        print(f"{i}. [{lab['charttime']}] {lab['label']}: {value_str}")

print("\n4. PROCEDURES")
print("-" * 80)
patient_procedures = [p for p in procedures if p["subject_id"] == example_pid]
print(f"Total procedures: {len(patient_procedures)}\n")
for i, proc in enumerate(patient_procedures, 1):
    print(f"Procedure {i}:")
    print(f"  Code: {proc['icd_code']}")
    print(f"  Description: {proc['long_title']}")
    print(f"  Date: {proc['chartdate']}")

print("\n5. DIAGNOSES (Past 2 Years - Top 10)")
print("-" * 80)
patient_conditions = [c for c in conditions if c["subject_id"] == example_pid]
print(f"Total diagnoses: {len(patient_conditions)}\n")

recent_hadm_ids = [e['hadm_id'] for e in patient_encounters]
recent_conditions = [c for c in patient_conditions if c['hadm_id'] in recent_hadm_ids]

condition_counts = Counter(c['long_title'] for c in recent_conditions if c['long_title'])
total_conditions = len(recent_conditions)

print(f"Diagnoses (all encounters): {total_conditions}\n")
for i, (diagnosis, count) in enumerate(condition_counts.most_common(10), 1):
    pct = (count / total_conditions * 100) if total_conditions > 0 else 0
    print(f"{i}. {diagnosis}: {count} times ({pct:.1f}%)")

print("\n6. MEDICATIONS (Past 2 Years - Top 10)")
print("-" * 80)
patient_meds = [m for m in medications if m["subject_id"] == example_pid]
print(f"Total medication administrations: {len(patient_meds)}")

valid_meds = [m for m in patient_meds if m['charttime']]
if valid_meds:
    max_time = max(datetime.strptime(m['charttime'], '%Y-%m-%d %H:%M:%S') for m in valid_meds)
    cutoff_date = max_time - timedelta(days=2 * 365)
    recent_meds = [m for m in valid_meds if datetime.strptime(m['charttime'], '%Y-%m-%d %H:%M:%S') >= cutoff_date]

    med_counts = Counter(m['medication'] for m in recent_meds if m['medication'])
    total_recent = len(recent_meds)

    print(f"Medication administrations in past 2 years: {total_recent}\n")
    for i, (med, count) in enumerate(med_counts.most_common(10), 1):
        pct = (count / total_recent * 100) if total_recent > 0 else 0
        print(f"{i}. {med}: {count} administrations ({pct:.1f}%)")

print("\n" + "=" * 100)
print("END OF REPORT")
print("=" * 100)

PATIENT SUMMARY REPORT
Patient ID: 24450f28-a039-57d8-95c9-d7ba5508ecd4

1. PATIENT DEMOGRAPHICS
--------------------------------------------------------------------------------
id: 24450f28-a039-57d8-95c9-d7ba5508ecd4
gender: female
birth_date: 2119-12-30
marital_status: D
race: White
ethnicity: Not Hispanic or Latino

2. ADMISSIONS
--------------------------------------------------------------------------------
Total admissions: 3

Admission 1:
  hadm_id: 0a83714b-0c0e-5761-b9ea-3ff6d5fd5c8b
  admittime: 2148-03-07 23:30:00
  dischtime: 2148-03-13 14:50:00
  class: OBSENC, priority: R
Admission 2:
  hadm_id: 0071a339-74cd-596a-9083-771d41d6d118
  admittime: 2147-12-30 08:40:00
  dischtime: 2148-01-11 17:55:00
  class: EMER, priority: EM
Admission 3:
  hadm_id: 7e8c1a0f-0a7b-5b91-a351-0a10939deace
  admittime: 2148-01-25 22:58:00
  dischtime: 2148-01-30 11:23:00
  class: OBSENC, priority: R

3. LAB RESULTS (Past 2 Years - Top 10)
-------------------------------------------------------